In [182]:
from sklearn.feature_extraction.text import CountVectorizer
import re
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Importing Word_List

In [184]:
l1 = []
with open(r"C:\Users\khush\Python, 12-7\words_250000_train.txt","r") as file:
    words = file.read().splitlines()
    l1.append(words)
    file.close()
all_words = l1[0]
print(f"Number of words in the game: {len(all_words)}")

Number of words in the game: 227300


# Original Hangman Game

In [7]:
import random

class HangmanGame:
    def __init__(self, word_list):
        self.word_list = word_list
        self.target_word = ""
        self.guesses_left = 6
        self.guessed_letters = set()
        self.word_in_progress = []

    def choose_word(self):
        self.target_word = random.choice(self.word_list)
        self.word_in_progress = ["_"] * len(self.target_word)

    def display_word(self):
        return " ".join(self.word_in_progress)

    def make_guess(self, letter):
        if letter in self.guessed_letters:
            return "You already guessed that letter!"

        self.guessed_letters.add(letter)

        if letter in self.target_word:
            for i in range(len(self.target_word)):
                if self.target_word[i] == letter:
                    self.word_in_progress[i] = letter
            if "_" not in self.word_in_progress:
                return "Congratulations! You guessed the word: {}".format(self.target_word)
        else:
            self.guesses_left -= 1
            if self.guesses_left == 0:
                return "Sorry, you ran out of guesses. The word was: {}".format(self.target_word)

        return self.display_word() + f"\nLives left: {self.guesses_left}"

# Example usage:
word_list = all_words

custom_hangman = HangmanGame(word_list)
custom_hangman.choose_word()

print("Welcome to Hangman!")
print("Word to guess:", custom_hangman.display_word())

while custom_hangman.guesses_left > 0 and "_" in custom_hangman.word_in_progress:
    guess = input("Enter your guess: ").lower()
    result = custom_hangman.make_guess(guess)
    print(result)

print("Game over.")


Welcome to Hangman!
Word to guess: _ _ _ _ _ _ _ _ _
Enter your guess: c
c _ _ _ _ _ _ _ _
Lives left: 6
Enter your guess: h
c h _ _ _ _ _ _ _
Lives left: 6
Enter your guess: a
c h a _ _ _ _ _ _
Lives left: 6
Enter your guess: e
c h a _ _ e _ _ e
Lives left: 6
Enter your guess: l
c h a l l e _ _ e
Lives left: 6
Enter your guess: n
c h a l l e n _ e
Lives left: 6
Enter your guess: g
Congratulations! You guessed the word: challenge
Game over.


## Model - 1 -- cosine similarity

In [195]:
import random

class HangmanGame:
    def __init__(self, word_list):
        self.word_list = word_list
        self.target_word = ""
        self.guesses_left = 10
        self.guessed_letters = set()
        self.word_in_progress = []
        self.common_letters = ''
        

    def choose_word(self):
        self.target_word = random.choice(self.word_list)
        self.word_in_progress = ["_"] * len(self.target_word)

    def display_word(self):
        return " ".join(self.word_in_progress)
    
    
    def guess_here(self):
        
        #Guessing the first word using random guess, based on MOST COMMON LETTERS
        random_guess_char = "eariotns"
        """
        Most Common Letters in percentage
        E – 11.1607%
        A – 8.4966%
        R – 7.5809%
        I – 7.5448%
        O – 7.1635%
        T – 6.9509%
        N – 6.6544%
        S – 5.7351%
        
        """
        if len(re.findall(r"[a-zA-Z]+","".join(self.word_in_progress)))== 0:
            for char in range(len(random_guess_char)):
                if random_guess_char[char] in self.common_letters:
                    continue
                else:
                    self.common_letters += random_guess_char[char]
                    left_guesses = self.guesses_left
                    return random_guess_char[char]
        
        #predicted_char = len(re.findall(r"[a-zA-Z]+","".join(self.word_in_progress)))

        #first take the incomplete word from word_in_progress
        incomplete_word = "".join(self.word_in_progress)
        #Find all the same length words from the final_list to predict the next letter
        words_with_same_length = [char for char in self.word_list if len(char) == len(self.word_in_progress)]
        
        # Convert them into vectors using CountVectorizer
        vectorizer = CountVectorizer(analyzer = 'char')
        
        words_with_same_length_vector = vectorizer.fit_transform(words_with_same_length)
        
        incomplete_word_vector = vectorizer.transform([incomplete_word])
        similarities = cosine_similarity(words_with_same_length_vector, incomplete_word_vector)
        
        similar_dic = {}
        for i,j in enumerate(similarities.tolist()):
            similar_dic[i] = j[0]

        sorted_list = sorted(similar_dic.items(),key = lambda x:x[1],reverse = True)

        similar_values = [words_with_same_length[sorted_list[i][0]] for i in range(len(sorted_list))]
        
        def count_sorted_list(word):
            new_dic = {}
            for i in word:
                if i not in new_dic:
                    new_dic[i] = 1
                else:
                    new_dic[i] +=1
            new_list = list(dict((sorted(new_dic.items(),reverse = True,key = lambda x:x[1]))).keys())
            return new_list
        
        previous_guesses_left = self.guesses_left

        while True:
            current_guesses_left = self.guesses_left  # Record current value
            # logic to check guessed letters and return i for each similar_value
            for similar_word in similar_values:
                for i in count_sorted_list(similar_word):
                    if i not in self.guessed_letters:
                        
                        if current_guesses_left != previous_guesses_left:
                            
                            break  # Break the loop if guesses_left changes
                        return i
                if current_guesses_left == previous_guesses_left:
                    previous_guesses_left = current_guesses_left  # Update previous value
                else:
                    break  # Break if guesses_left changes


        
        
        

    def make_guess(self, letter):
        if letter in self.guessed_letters:
            return "You already guessed that letter!"

        self.guessed_letters.add(letter)

        if letter in self.target_word:
            for i in range(len(self.target_word)):
                if self.target_word[i] == letter:
                    self.word_in_progress[i] = letter
            if "_" not in self.word_in_progress:
                return "Congratulations! You guessed the word: {}".format(self.target_word)
        else:
            self.guesses_left -= 1
            if self.guesses_left == 0:
                return "Sorry, you ran out of guesses. The word was: {}".format(self.target_word)

        return self.display_word() + f"\nLives left: {self.guesses_left}"

# Example usage:
word_list = all_words
#word_list = ['aion']
custom_hangman = HangmanGame(word_list)
custom_hangman.choose_word()

print("Welcome to Hangman!")
print("Word to guess:", custom_hangman.display_word())

while custom_hangman.guesses_left > 0 and "_" in custom_hangman.word_in_progress:
    #guess = input("Enter your guess: ").lower()
    guess = custom_hangman.guess_here()
    result = custom_hangman.make_guess(guess)
    print(result)

print("Game over.")


Welcome to Hangman!
Word to guess: _ _ _ _ _ _ _ _
_ e _ e _ _ _ _
Lives left: 10
_ e _ e _ _ _ _
Lives left: 9
_ e _ e _ _ _ _
Lives left: 8
l e _ e _ _ _ _
Lives left: 8
l e _ e _ _ _ _
Lives left: 7
l e _ e _ _ _ _
Lives left: 6
l e _ e _ _ _ _
Lives left: 5
l e _ e _ _ _ _
Lives left: 4
l e _ e _ _ _ c
Lives left: 4
l e _ e _ _ _ c
Lives left: 3
l e _ e _ _ _ c
Lives left: 2
l e _ e _ d _ c
Lives left: 2
l e _ e _ d _ c
Lives left: 1
Sorry, you ran out of guesses. The word was: legendic
Game over.


Number of words in the game: 227300


In [170]:
all_words = l1[0]

In [169]:
len(l1[0])

227300

In [130]:
def count_sorted_list(word):
    new_dic = {}
    for i in word:
        if i not in new_dic:
            new_dic[i] = 1
        else:
            new_dic[i] +=1
    new_dic = list(dict((sorted(new_dic.items(),reverse = True,key = lambda x:x[1]))).keys())
    return new_dic

d1 = count_sorted_list("hangman")

d1      

['a', 'n', 'h', 'g', 'm']

In [116]:
d1[1]

KeyError: 1

In [41]:
word = 'hangmn'
word_in_progress = ["_"] * 6
word_in_progress

['_', '_', '_', '_', '_', '_']

In [110]:
word_list = ["pythona", "hangman", "codinga", "challea", "gamedaa"]

word_in_progress = ["_"] * 7
word_in_progress[1] = "y"


incomplete_word = "".join(word_in_progress)
#Find all the same length words from the final_list to predict the next letter
words_with_same_length = [char for char in word_list if len(char) == len(word_in_progress)]

# Convert them into vectors using CountVectorizer
vectorizer = CountVectorizer(analyzer = 'char')
words_with_same_length_vector = vectorizer.fit_transform(words_with_same_length)
incomplete_word_vector = vectorizer.transform([incomplete_word])


similarities = cosine_similarity(words_with_same_length_vector, incomplete_word_vector)

similar_dic = {}
for i,j in enumerate(similarities.tolist()):
    similar_dic[i] = j[0]
    
sorted_list = sorted(similar_dic.items(),key = lambda x:x[1],reverse = True)

similar_values = [words_with_same_length[sorted_list[i][0]] for i in range(len(sorted_list))]
print(similar_values)

['pythona', 'hangman', 'codinga', 'challea', 'gamedaa']


In [107]:
for i in range(len(sorted_list)):
    print(words_with_same_length[sorted_list[i][0]])

challea
hangman
gamedaa
pythona
codinga


In [108]:
[words_with_same_length[sorted_list[i][0]] for i in range(len(sorted_list))]

['challea', 'hangman', 'gamedaa', 'pythona', 'codinga']

In [101]:
words_with_same_length

['pythona', 'hangman', 'codinga', 'challea', 'gamedaa']

In [96]:
similar_dic

{0: 0.3779644730092272,
 1: 0.30151134457776363,
 2: 0.0,
 3: 0.30151134457776363,
 4: 0.0}

In [97]:
words_with_same_length

['pythona', 'hangman', 'codinga', 'challea', 'gamedaa']

In [93]:
sorted_list

[(0, 0.30151134457776363), (1, 0.3333333333333333)]

In [99]:
sorted_list

[(0, 0.3779644730092272),
 (1, 0.30151134457776363),
 (3, 0.30151134457776363),
 (2, 0.0),
 (4, 0.0)]

In [49]:
similarities = np.vstack([similarities, np.array([0.1])])
similarities

array([[0.28867513],
       [0.57735027],
       [0.1       ]])

In [71]:
similar_dic = {}
for i,j in enumerate(similarities.tolist()):
    similar_dic[i] = j[0]
    
similar_dic

{0: 0.2886751345948129, 1: 0.5773502691896258}

In [69]:
sorted(similar_dic.items(),key = lambda x:x[1])

[(2, 0.1), (0, 0.2886751345948129), (1, 0.5773502691896258)]

In [72]:
words_with_same_length[sorted(similar_dic.items(),key = lambda x:x[1])[0][0]]

'python'

In [50]:
sorted(similarities.tolist())

[[0.1], [0.2886751345948129], [0.5773502691896258]]

In [25]:
words_with_same_length

['python', 'coding']

In [33]:
np.sort(similarities,axis = 0)[::-1]

array([[0.57735027],
       [0.28867513]])

In [41]:
np.argsort(similarities,axis = 0,kind = 'mergesort')

array([[4],
       [0],
       [1],
       [2],
       [3]], dtype=int64)